In [0]:
# Load the Drive helper and mount 
from google.colab import drive
# # This will prompt for authorization.
drive.mount('/content/drive/')
# #Navigate to the directory containing this notebook
%cd "/content/drive/My Drive/StarliperSongkakul-Project3/Dense2Net"


!ls

In [5]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
#from .utils import load_state_dict_from_url
from collections import OrderedDict

import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import time
import sys
import numpy as np
import pickle as pkl
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


__all__ = ['DenseNet', 'densenet121', 'densenet169', 'densenet201', 'densenet161']

model_urls = {
    'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
    'densenet169': 'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
    'densenet201': 'https://download.pytorch.org/models/densenet201-c1103571.pth',
    'densenet161': 'https://download.pytorch.org/models/densenet161-8d451a50.pth',
}


class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate,
                                bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=100): #TANNER: changed drop rate to 0.2 from 0

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate,
                                drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out


def _load_state_dict(model, model_url, progress):
    # '.'s are no longer allowed in module names, but previous _DenseLayer
    # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
    # They are also in the checkpoints in model_urls. This pattern is used
    # to find such keys.
    pattern = re.compile(
        r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')

    state_dict = load_state_dict_from_url(model_url, progress=progress)
    for key in list(state_dict.keys()):
        res = pattern.match(key)
        if res:
            new_key = res.group(1) + res.group(2)
            state_dict[new_key] = state_dict[key]
            del state_dict[key]
    model.load_state_dict(state_dict)


def _densenet(arch, growth_rate, block_config, num_init_features, pretrained, progress,
              **kwargs):
    model = DenseNet(growth_rate, block_config, num_init_features, **kwargs)
    if pretrained:
        _load_state_dict(model, model_urls[arch], progress)
    return model


def densenet121(pretrained=False, progress=True, **kwargs):
    r"""Densenet-121 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet121', 32, (6, 12, 24, 16), 64, pretrained, progress,
                     **kwargs) #TANNER: changed growth rate to 12 from 32


def densenet161(pretrained=False, progress=True, **kwargs):
    r"""Densenet-161 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet161', 48, (6, 12, 36, 24), 96, pretrained, progress,
                     **kwargs)


def densenet169(pretrained=False, progress=True, **kwargs):
    r"""Densenet-169 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet169', 32, (6, 12, 32, 32), 64, pretrained, progress,
                     **kwargs)


def densenet201(pretrained=False, progress=True, **kwargs):
    r"""Densenet-201 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet201', 32, (6, 12, 48, 32), 64, pretrained, progress,
                     **kwargs)
      
def conv3x3(in_planes, out_planes, stride=1, groups=1):  
    #returns a 3x3 2d convolution, used in Res2Net sub-convolutions
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, groups=groups, bias=False)

class Res2Net_block(nn.Module):
    #Res2net bottleneck block
    def __init__(self, planes, scale=1, stride=1, groups=1, norm_layer=None):
        super(Res2Net_block, self).__init__()
        
        self.relu = nn.ReLU(inplace=True)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
            
        self.scale = scale
        ch_per_sub = planes // self.scale
        ch_res = planes % self.scale    
        self.chunks  = [ch_per_sub * i + ch_res for i in range(1, scale + 1)]
        self.conv_blocks = self.get_sub_convs(ch_per_sub, norm_layer, stride, groups)
        
    def forward(self, x):
        sub_convs = []
        sub_convs.append(x[:, :self.chunks[0]])
        sub_convs.append(self.conv_blocks[0](x[:, self.chunks[0]: self.chunks[1]]))
        for s in range(2, self.scale):
            sub_x = x[:, self.chunks[s-1]: self.chunks[s]]
            sub_x += sub_convs[-1]
            sub_convs.append(self.conv_blocks[s-1](sub_x))

        return torch.cat(sub_convs, dim=1)
    
    def get_sub_convs(self, ch_per_sub, norm_layer, stride, groups):
        layers = []
        for _ in range(1, self.scale):
            layers.append(nn.Sequential(
                conv3x3(ch_per_sub, ch_per_sub, stride, groups),
                norm_layer(ch_per_sub), self.relu))
        
        return nn.Sequential(*layers)

      
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    overfit = 0
    overfit_limit = 99.98
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    avg_loss = 0
    acc = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100. * correct / total
        avg_loss = train_loss/(batch_idx+1)
        if(batch_idx%25==0) or batch_idx==len(trainloader)-1:
          print(batch_idx+1, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (avg_loss, acc, correct, total))
    
    if acc>overfit_limit:
        overfit=1
        
    return avg_loss, acc, overfit


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    avg_loss = 0
    acc = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            acc = 100. * correct / total
            avg_loss = test_loss / (batch_idx + 1)
            if(batch_idx%25==0) or batch_idx==len(testloader)-1:
                print(batch_idx+1, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.t7')
        best_acc = acc
    return avg_loss, acc


def plot_curves(train, test, mode):
    if mode == 1:
        title = "Accuracy Curves"
        label = "Accuracy"
        filename = "acc.png"
    else:
        title = "Loss Curves"
        label = "Loss"
        filename = "loss.png"
    num_epochs_plot = range(0, len(train))  # x axis range
    plt.figure()
    plt.plot(num_epochs_plot, train, "b", label="Training")
    plt.plot(num_epochs_plot, test, "r", label="Validation")
    plt.title(title)
    plt.xlabel("Number of Epochs")
    plt.ylabel(label)
    plt.legend()
    plt.savefig(filename)
    plt.close()

if __name__ == '__main__':

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    best_acc = 0  # best test accuracy
    start_epoch = 0  # start from epoch 0 or last checkpoint epoch
    end_epoch = 100 # number of epochs to run
    augment = 1 # set to 1 to augment data
    resume = 0 # resume from checkpoint 

    # Data
    print('==> Preparing data..')
    if augment==0: #TANNER: this was backwards before, switched in v2
      transform_train = transforms.Compose([
          transforms.ToTensor(),
          #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #CIFAR10
          transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276]), #CIFAR100

      ])
    else:
      transform_train = transforms.Compose([
          transforms.RandomCrop(32, padding=4),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #CIFAR10
          transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276]), #CIFAR100
      ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #CIFAR10
        transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276]), #CIFAR100
    ])

    trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

    # classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    # Model
    print('==> Building model..')
    net = densenet121(False)
    net = net.to(device)
    if device == 'cuda':
        net = nn.DataParallel(net)
        cudnn.benchmark = True
    
    learning_rate = 0.1
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001) #(tanner): changed weight decay to 0.0001 to match Res2net paper


    train_loss = np.zeros((end_epoch, 1))
    train_acc = np.zeros((end_epoch, 1))
    test_acc = np.zeros((end_epoch, 1))
    test_loss = np.zeros((end_epoch, 1))

    if resume == 1:
	    # Load checkpoint.
	    print('==> Resuming from checkpoint..')
	    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
	    checkpoint = torch.load('./checkpoint/ckpt.t7')
	    net.load_state_dict(checkpoint['net'])
	    best_acc = checkpoint['acc']
	    start_epoch = checkpoint['epoch']
	    with open('results.pkl', 'rb') as file:
      		train_loss, train_acc, test_loss, test_acc = pkl.load(file)

    track_overfit = 0
    #train and test
    start_time = time.time()
    last_epoch = end_epoch
    for epoch in range(start_epoch, end_epoch):
      if (epoch == 50) or (epoch == 75):#((epoch+1)%30)==0:
          for g in optimizer.param_groups:
            learning_rate /= 10
            g['lr'] = learning_rate #reduce learning rate by a factor of 10 every 30 epochs
            print('learning rate reduced to '+str(learning_rate))

      train_loss[epoch], train_acc[epoch],overfit_check = train(epoch)
      test_loss[epoch], test_acc[epoch] = test(epoch)
      results = [train_loss, train_acc, test_loss, test_acc]
      with open('results.pkl', 'wb') as file:
          pkl.dump(results, file)
      track_overfit+= overfit_check
      if track_overfit>3:
          print('overfit, terminating run')
          last_epoch=epoch
          break
    #plot training and testing curves
    plot_curves(train_loss[0:last_epoch+1], test_loss[0:last_epoch+1], 0)
    plot_curves(train_acc[0:last_epoch+1], test_acc[0:last_epoch+1], 1)
    end_time = time.time()
    print('Total Time: ', (end_time - start_time))


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..

Epoch: 0
1 391 Loss: 4.712 | Acc: 0.781% (1/128)
26 391 Loss: 4.966 | Acc: 3.185% (106/3328)
51 391 Loss: 4.913 | Acc: 3.554% (232/6528)
76 391 Loss: 4.827 | Acc: 4.019% (391/9728)
101 391 Loss: 4.732 | Acc: 4.494% (581/12928)
126 391 Loss: 4.623 | Acc: 5.066% (817/16128)
151 391 Loss: 4.536 | Acc: 5.443% (1052/19328)
176 391 Loss: 4.463 | Acc: 5.842% (1316/22528)
201 391 Loss: 4.402 | Acc: 6.281% (1616/25728)
226 391 Loss: 4.347 | Acc: 6.568% (1900/28928)
251 391 Loss: 4.294 | Acc: 6.975% (2241/32128)
276 391 Loss: 4.246 | Acc: 7.391% (2611/35328)
301 391 Loss: 4.199 | Acc: 7.841% (3021/38528)
326 391 Loss: 4.159 | Acc: 8.244% (3440/41728)
351 391 Loss: 4.122 | Acc: 8.551% (3842/44928)
376 391 Loss: 4.087 | Acc: 8.897% (4282/48128)
391 391 Loss: 4.064 | Acc: 9.196% (4598/50000)
1 100 Loss: 3.908 | Acc: 9.000% (9/100)
26 100 Loss: 3.840 | Acc: 15.769% (410/2600)
51 100

KeyboardInterrupt: ignored